# Lecture 8: Iterators and generators

- Implement iterator behavior in custom classes so they can be looped through using the same convenient syntax as built-in containers
- Use generators to create simple, custom iterators

__Reading material__: 
- [Python tutorial](https://docs.python.org/3.7/tutorial/) 9.8 - 9.10
- Optional: http://anandology.com/python-practice-book/iterators.html

We’re learning to write the blueprint for our own container objects, that is, objects that contain multiple elements that we can access individually and iterate over in a for loop. Python lists, tuples, sets, and dictionaries are all built-in container objects. 

In [1]:
for element in [1, 2, 3]:
    print(element)
for element in (1, 2, 3):
    print(element)
for key in {'one':1, 'two':2}:
    print(key)
for char in "123":
    print(char)

1
2
3
1
2
3
one
two
1
2
3


In [2]:
myList = [1, 2, 3]

In [4]:
#dir(myList) shows available attributes
myIterator = iter(myList)

In [5]:
type(myIterator)

list_iterator

In [7]:
# dir(myIterator)
print(next(myIterator))

1


In [8]:
print(next(myIterator))

2


In [9]:
print(next(myIterator))

3


In [10]:
print(next(myIterator))

StopIteration: 

In [11]:
# first creates an iterator by calling iter(myList)
# repeatedly calls the next method on this iterator until it raises StopIteration
for elem in myList:
    print(elem)

1
2
3


## Iterators
Now we’re trying to create our own.
Our custom container might have a built-in container object as an instance variable. For instance, the following __Reverse__ class has an instance variable data that is already a container. In this case, we just need to write some code that tells a for loop how to iterate over that instance variable.

In [12]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data

    def __iter__(self):
        return ReverseIterator(self)
    
class ReverseIterator:
    def __init__(self, reverseObject):
        self.index = len(reverseObject.data)
        self.r = reverseObject
        
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.r.data[self.index]

In [17]:
rev = Reverse('spam')

In [14]:
print(rev.data)

spam


In [18]:
revIter = iter(rev)

In [19]:
print(next(revIter))

m


In [20]:
print(revIter.r, revIter.index)

<__main__.Reverse object at 0x1076ff7d0> 3


In [22]:
print(next(revIter))
print(revIter.r, revIter.index)

a
<__main__.Reverse object at 0x1076ff7d0> 2


In [23]:
print(next(revIter))
print(revIter.r, revIter.index)

p
<__main__.Reverse object at 0x1076ff7d0> 1


In [24]:
print(next(revIter))
print(revIter.r, revIter.index)

s
<__main__.Reverse object at 0x1076ff7d0> 0


In [25]:
print(next(revIter))

StopIteration: 

In [16]:
for char in rev:
    print(char)

m
a
p
s


- The simplest sort of container object will have its own `__next__` method that, when called, returns to the for loop the next element in the container. When there are no more elements in the container, it raises a StopIteration exception (see 8.4 in [Python tutorial](https://docs.python.org/3.7/tutorial/) ) instead of returning an element. The for loop terminates when it gets this exception.

- In general, however, the container object doesn’t need to have its own `__next__` method. Instead, it may assign the job of picking the next element to a separate object, called an __iterator__.

- In general, an iterator is any object that defines a suitable `__next__` method. When an iterator object’s `__next__` method is invoked, the method should return the next element of some collection – whatever that may mean. How the `__next__` method is written defines the order in which the elements of a collection are iterated over in a for loop.

- Your collection appoints an iterator by defining an `__iter__` method that returns an instance of an iterator object.

If the collection has its own next method, the collection’s `__iter__` method can return self; the container will serve as its own iterator. Note that in the following example, the __Reverse__ class is both the container and the iterator object. But in general, the iterator can be a separate object from the container.

In [27]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [28]:
rev = Reverse('apple')
iter(rev)
for char in rev:
    print(char)

e
l
p
p
a


In [29]:
rev = Reverse('apple')
revIter = iter(rev)
print(rev is revIter)

True


Here's another example:

In [30]:
class ThreeElementContainer:
    def __init__(self, a = 0, b = 0, c = 0):
        self.a = a
        self.b = b
        self.c = c
        self.i = 0
    
    def __iter__(self):
        print("iter called")
        return self
        
    def __next__(self):
        print("next called")
        if self.i == 0:
            el = self.a
        elif self.i == 1:
            el = self.b
        elif self.i == 2:
            el = self.c
        else:
            print("raised an exception")
            raise StopIteration
        self.i += 1
        return el
        
    def __str__(self):
        return "[" + str(self.a) + ", " + str(self.b) + ", " + str(self.c)+ "]"
        
t = ThreeElementContainer(5,10,15)
print(t)
# for el in t:
#     print(el)

[5, 10, 15]


In [31]:
for el in t:
    print(el)

iter called
next called
5
next called
10
next called
15
next called
raised an exception


In [32]:
for el in t:
    print(el)

iter called
next called
raised an exception


In [34]:
class ThreeElementContainer2:
    def __init__(self, a = 0, b = 0, c = 0):
        self.a = a
        self.b = b
        self.c = c
        self.i = 0
    
    def __iter__(self):
        print("iter called")
        self.i = 0 # reset self.i
        return self
        
    def __next__(self):
        print("next called")
        if self.i == 0:
            el = self.a
        elif self.i == 1:
            el = self.b
        elif self.i == 2:
            el = self.c
        else:
            print("raised an exception")
            raise StopIteration
        self.i += 1
        return el
        
    def __str__(self):
        return "[" + str(self.a) + ", " + str(self.b) + ", " + str(self.c)+ "]"
        
t = ThreeElementContainer2(5,10,15)
print(t)

[5, 10, 15]


In [35]:
for el in t:
    print(el)

iter called
next called
5
next called
10
next called
15
next called
raised an exception


In [36]:
for el in t:
    print(el)

iter called
next called
5
next called
10
next called
15
next called
raised an exception


## Generators
The generator is the elegant brother of iterator that allows you to write iterators like the one you saw earlier, but in a much easier syntax where you do not have to write classes with `__iter__` and `__next__` methods.

Now let's write a __generator__ every_other(data) that yields every other
element of the data. 

In [37]:
def every_other(data):
    for index in range(0,len(data),2):
        yield data[index]
        
# for char in every_other("supercalifragilisticexpialidocious"):
#     print(char, end=" ")

In [44]:
for char in every_other("supercalifragilisticexpialidocious"):
    print(char, end=" ")

s p r a i r g l s i e p a i o i u 

In [38]:
f = every_other("supercalifragilisticexpialidocious")

In [39]:
print(type(f))

<class 'generator'>


In [41]:
next(f)

's'

In [42]:
next(f)

'p'

In [43]:
next(f)

'r'

The magic word with generators is `yield`. There is no return statement in the function `every_other`. The return value of the function will actually be a generator. Inside the for loop when the execution reaches the yield statement, the value of data[index] is returned and the generator state is suspended. During the second next call, the generator resumes from the index at which it stopped earlier and increases this index by one. It continues with the for loop and comes to the yield statement again.

`yield` basically replaces the return statement of a function but rather provides a result to its caller without destroying local variables. Thus, in the next iteration, it can work on this local variable value again. So unlike a normal function that you have seen before, where on each call it starts with new set of variables - a generator will resume the execution where it was left off.

Now let's rewrite the ThreeElementContainer class using a generator:

In [3]:
class ThreeElementContainer:
    def __init__(self, a = 0, b = 0, c = 0):
        self.a = a
        self.b = b
        self.c = c
#        self.i = 0
    
#    def __iter__(self):
#        print("iter called")
#        return self

    def __iter__(self):
        return self.generator()
    
    def generator(self):
        yield self.a
        yield self.b
        yield self.c
        
#    def next(self):
#        print("next called")
#        if self.i == 0:
#            el = self.a
#        elif self.i == 1:
#            el = self.b
#        elif self.i == 2:
#            el = self.c
#        else:
#            print("raised an exception")
#            raise StopIteration
#        self.i += 1
#        return el
        
    def __str__(self):
        return "[" + str(self.a) + ", " + str(self.b) + ", " + str(self.c)+ "]"
        
t = ThreeElementContainer(5,10,15)
# print(t)
for el in t:
    print(el)
for el in t:
    print(el)

5
10
15
5
10
15


## Generator expressions
The generator expressions are the generator equivalent of a list comprehension, but with parentheses instead of square brackets. Just like a list comprehension returns a list, a generator expression will return a generator.

In [46]:
squares = (x * x for x in range(1,10))
print(type(squares))
# print(list(squares))

<class 'generator'>


We can also create the same generator every_other (as above) in a single line using a
lambda function and a generator expression.

In [47]:
every_other2 = lambda data: (char for char in data[::2])
for char in every_other2("supercalifragilisticexpialidocious"):
    print(char, end=" ")

print("")

# a more memory-efficient solution
every_other3 = lambda data: (data[index] for index in range(0,len(data),2)) # range would work, too
for char in every_other3("supercalifragilisticexpialidocious"):
    print(char, end=" ")

s p r a i r g l s i e p a i o i u 
s p r a i r g l s i e p a i o i u 